# Дипломный проект  
## Определение уязвимых групп населения и кластеризация регионов России  

### Часть 1. Подготовка, очистка и объединение данных.

**Описание проекта**

Цель данного проекта:
- кластеризовать регионы России и определить, какие из них наиболее остро нуждаются в помощи малообеспеченным/неблагополучным слоям населения;
- описать группы населения, сталкивающиеся с бедностью;
- определить факторы, влияющие на уровень бедности.

Для этого собираются и объединяются данные Росстата по ключевым направлениям, отражающим благосостояние населения, развитие экономики и социальную напряжённость. Перечень показателей включает данные о доходах, бедности, промышленном производстве, структуре инвалидности, рождаемости, доступности жилья, алкогольных и наркотических психозах, занятости, региональном продукте и другие.

**В этой части проекта выполняется**
 
1. Загрузка всех исходных таблиц (форматы csv, xls, xlsx).
2. Очистка данных.
3. Приведение таблиц к единому виду:
region | year | <показатели...>.
4. Объединение всех таблиц в одну итоговую панель данных.
5. Описание каждой таблицы и её показателей (структура, единицы измерения, смысловая интерпретация).
6. Сохранение итогового датасета для дальнейшего анализа.

**Результатом** этой части является единый очищенный датасет, готовый для:

- анализа распределений и взаимосвязей показателей (EDA),
- нормализации данных,
- построения кластеров регионов.

In [1]:
# Подключаем необходимые для работы библиотеки
import pandas as pd
import numpy as np
import xlrd
import re
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#### Вспомогательные функции для унификации и предобработки данных

Данный раздел включает функции, обеспечивающие предварительную очистку и стандартизацию таблиц, а также преобразование данных в формат, необходимый для последующего анализа.

In [2]:
def clean_regions(df, region_col='region'):
    """
    Унифицирует названия регионов.

    Делает:
    - удаление переносов строк
    - удаление лишних пробелов
    - латинская B в кириллическую В
    - приведение к единому виду согласно словарю соответствий
    - удаление строк с федеральными округами и РФ

    Возвращает новый датафрейм (копию).
    """
    # Убираем явные NaN в колонке региона
    df = df[df[region_col].notna()].copy()

    # Приводим к строке и чистим
    df[region_col] = df[region_col].astype(str).str.strip()
    df[region_col] = df[region_col].str.replace("\n", " ", regex=False)
    df[region_col] = df[region_col].str.replace(r"^B", "В", regex=True)
    df[region_col] = df[region_col].str.replace(r"\s+", " ", regex=True)

    # Словарь ручных переименований регионов
    region_fix_map = {
        "Кемеровская область - Кузбасс": "Кемеровская область",
        "Кемеровская область – Кузбасс": "Кемеровская область",
        "Город Москва столица Российской Федерации город федерального значения": "г. Москва",
        "Город Санкт-Петербург город федерального значения": "г. Санкт-Петербург",
        "Город федерального значения Севастополь": "г. Севастополь",
        "Чувашская Республика - Чувашия": "Чувашская Республика",
        "Ямало-Ненецкий автономный округ (Тюменская область)": "Ямало-Ненецкий автономный округ",
        "Ямало-Ненецкий автономный округ": "Ямало-Ненецкий автономный округ",
        "Ханты-Мансийский автономный округ - Югра (Тюменская область)": "Ханты-Мансийский автономный округ",
        "Ханты-Мансийский автономный округ – Югра": "Ханты-Мансийский автономный округ",
        "Республика Татарстан (Татарстан)": "Республика Татарстан",
        "Республика Адыгея (Адыгея)": "Республика Адыгея",
        "Ненецкий автономный округ (Архангельская область)": "Ненецкий автономный округ",
        "Ненецкий авт. округ": "Ненецкий автономный округ",
        "Архангельская область без авт. округа": "Архангельская область (кроме Ненецкого автономного округа)",
        "Архангельская область без автономного округа": "Архангельская область (кроме Ненецкого автономного округа)",
        "Еврейская авт. область": "Еврейская автономная область",
        "Республика Саха(Якутия)": "Республика Саха (Якутия)",
        "Тюменская область без авт. округов": "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)",
        "Тюменская область без автономных округов": "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)",
        "Ханты-Мансийский авт. округ": "Ханты-Мансийский автономный округ",
        "Чукотский авт. округ": "Чукотский автономный округ",
        "Ямало-Ненецкий авт. округ": "Ямало-Ненецкий автономный округ",
        "г.Москва": "г. Москва",
        "город Москва": "г. Москва",
        "город Санкт-Петербург": "г. Санкт-Петербург",
        "город Севастополь": "г. Севастополь",
        "Ханты-Мансийский автономный округ - Югра": "Ханты-Мансийский автономный округ",
        "Республика Северная Осетия – Алания": "Республика Северная Осетия-Алания"
    }
    
    df[region_col] = df[region_col].replace(region_fix_map)
    
    mask_country = df[region_col] == 'Российская Федерация'
    mask_fd = df[region_col].str.contains('федеральный округ', case=False, na=False)
    mask_empty = df[region_col] == ""

    df = df.loc[~(mask_country | mask_fd | mask_empty)].reset_index(drop=True)
    
    return df

In [3]:
def filter_years(df, year_col='year', start=2015, end=2020):
    """
    Оставляет строки с годами в заданном диапазоне [start, end].
    Возвращает отфильтрованный датафрейм.
    """
    df = df.query(f'{year_col} >= {start} and {year_col} <= {end}')
    
    return df

In [4]:
def wide_years_to_long(df, id_vars, value_name, year_col='year'):
    """
    Превращает таблицу с колонками-годами в long-формат.

    Находит все столбцы, названия которых - 4-значный год (например, '2015').
    Результат:
        id_vars... | year | value_name
    """
    # Выбираем только те колонки, которые выглядят как год
    year_cols = [c for c in df.columns if re.fullmatch(r'\d{4}', str(c))]

    df_long = df.melt(
        id_vars=id_vars,
        value_vars=year_cols,
        var_name=year_col,
        value_name=value_name
    )

    # Приведение типов
    df_long[year_col] = df_long[year_col].astype(int)
    df_long[value_name] = pd.to_numeric(df_long[value_name], errors='coerce')

    return df_long

In [5]:
def load_excel_region_years(path: str,
                            sheet_name=0,
                            header_row: int = 2,
                            region_col_index: int = 0,
                            value_name: str = "value"):
    """
    Универсальная функция для таблиц формата:
    - на 3-й строке (index=2) шапка;
    - одна из колонок - регион;
    - остальные колонки содержат годы (в названии есть 4 цифры).

    Делает:
    - читает Excel,
    - переименовывает колонку региона в 'region',
    - вытаскивает годы из названий колонок,
    - превращает таблицу в long-формат,
    - чистит регионы и ограничивает по годам.

    Возвращает:
        region | year | <value_name>
    """
    # Читаем Excel
    df = pd.read_excel(
        path,
        sheet_name=sheet_name,
        header=header_row
    )

    # Переименовываем колонку с регионом в 'region'
    region_col_name = df.columns[region_col_index]
    df = df.rename(columns={region_col_name: "region"})

    # Из остальных названий колонок вытаскиваем год (4 цифры)
    new_cols = []
    for col in df.columns:
        if col == "region":
            new_cols.append("region")
        else:
            text = str(col)
            match = re.search(r"(\d{4})", text)
            if match:
                new_cols.append(int(match.group(1)))   # год как int
            else:
                new_cols.append(col)  # если года нет - оставляем как есть

    df.columns = new_cols

    # Преобразовываем таблицу в нужный вид
    df = wide_years_to_long(
        df,
        id_vars=["region"],
        value_name=value_name,
        year_col="year"
    )

    # чистим регионы и ограничиваем годами
    df = clean_regions(df, region_col="region")
    df = filter_years(df, year_col="year")

    # финальный вид
    df = df[["region", "year", value_name]].reset_index(drop=True)

    return df

In [6]:
def load_excel_years(path: str,
                     value_name: str,
                     sheet_name=0,
                     header: int = 0):
    """
    Загружает таблицу Excel формата:
        region | <year1> | <year2> | ...

    и преобразует её в вид:
        region | year | <value_name>

    Делает:
    - читает Excel;
    - переименовывает первую колонку в 'region';
    - выбирает только колонки с годами (имя - только цифры);
    - в long через wide_years_to_long;
    - чистит названия регионов;
    - удаляет строки без значения показателя;
    - ограничивает период;
    - упорядочивает колонки.
    """

    # Читаем Excel
    df = pd.read_excel(
        path,
        sheet_name=sheet_name,
        header=header
    )

    # Переименовываем первую колонку в region
    first_col = df.columns[0]
    df = df.rename(columns={first_col: 'region'})

    # Чистим названия регионов
    df = clean_regions(df, region_col='region')

    # Определяем годовые колонки (имя колонки - только цифры)
    year_cols = [col for col in df.columns if str(col).isdigit()]

    # Оставляем только region + годы
    df = df[['region'] + year_cols].copy()

    # Приводим таблицу в нужный вид
    df_long = wide_years_to_long(
        df,
        id_vars=['region'],
        value_name=value_name,
        year_col='year'
    )

    # Удаляем строки без значения показателя
    df_long = df_long.dropna(subset=[value_name]).copy()

    # Фильтрация по годам
    df_long = filter_years(df_long, year_col='year')

    # Упорядочивание колонок
    df_long = df_long[['region', 'year', value_name]].reset_index(drop=True)

    return df_long


In [7]:
def load_crime_year_file(path: str, year: int):
    """
    Читает годовой файл по преступлениям
    и возвращает DataFrame с колонками:
    region, year, crime_...
    """
    df = pd.read_excel(
        path,
        skiprows=12,   # как раньше
        header=None,
        usecols="B:L"  # B..L
    )

    # Переименовываем колонки 
    df = df.rename(columns={
        1: "region",
        2: "crime_juveniles",          # несовершеннолетними или при их соучастии
        3: "crime_repeat_offenders",   # ранее совершавшими преступления
        4: "crime_prev_convicted",     # в том числе ранее судимыми
        5: "crime_group",              # группой лиц
        6: "crime_group_prep",         # группой лиц по предварительному сговору
        7: "crime_organized_group",    # организованной группой
        8: "crime_criminal_community", # преступным сообществом (преступной организацией)
        9: "crime_intox_alcohol",      # в состоянии опьянения: алкогольного
        10: "crime_intox_narcotic",    # наркотического
        11: "crime_intox_toxic",       # токсического
    })

    # Добавляем год
    df["year"] = year
    df = clean_regions(df, region_col="region")

    # Финальный порядок колонок
    cols = [
        "region", "year",
        "crime_juveniles",
        "crime_repeat_offenders",
        "crime_prev_convicted",
        "crime_group",
        "crime_group_prep",
        "crime_organized_group",
        "crime_criminal_community",
        "crime_intox_alcohol",
        "crime_intox_narcotic",
        "crime_intox_toxic",
    ]
    df = df[cols]

    return df

#### 1. Бедность населения

Процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам.

**Файл**: poverty_percent_by_regions_1992_2020.csv

**Единица измерения**: процент (% населения региона).

**Описание исходных данных**: таблица содержит официальные данные Росстата о доле населения, располагающего денежными доходами ниже величины прожиточного минимума в каждом субъекте Федерации.

Показатель публикуется ежегодно и отражает уровень абсолютной бедности в регионе: тех жителей, чей объем доступных денежных ресурсов недостаточен для обеспечения минимальных потребностей.

**Первоначальная таблица** включает:

названия регионов;
год наблюдения;
показатель poverty_percent - доля бедного населения в процентах.

После очистки и приведения данных к единому формату **итоговая таблица** имеет вид:

| Признаки            | Описание                                                        |
| ------------------- | --------------------------------------------------------------- |
| **region**          | субъект Российской Федерации                                    |
| **year**            | год наблюдения                                                  |
| **poverty_percent** | доля населения региона с доходами ниже прожиточного минимума, % |

Период, используемый в проекте: 2015-2020 гг.

Показатель бедности является одним из ключевых индикаторов социально-экономического неблагополучия региона.

In [8]:
poverty_path = '../data_raw/poverty_percent_by_regions_1992_2020.csv'

poverty = pd.read_csv(poverty_path)

# Чистим регионы
poverty = clean_regions(poverty, region_col='region')

# Фильтруем годы
poverty = filter_years(poverty, year_col='year')

# Переупорядочиваем колонки
poverty = poverty[['region', 'year', 'poverty_percent']].reset_index(drop=True)

print('Годы в poverty:', sorted(poverty['year'].unique()))
display(poverty.head(5))

Годы в poverty: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,poverty_percent
0,Белгородская область,2015,8.5
1,Белгородская область,2016,8.1
2,Белгородская область,2017,7.8
3,Белгородская область,2018,7.5
4,Белгородская область,2019,7.8


#### 2. Население.
   
Численность постоянного населения по регионам и федеральным округам на 1 января каждого года.

**Файл**: population.xlsx
   
**Единица измерения**: человек.

**Описание исходных данных**: таблица содержит официальные данные Росстата о численности постоянного населения каждого субъекта Российской Федерации на 1 января соответствующего года.

Тип сбора - «накопленный итог», что означает, что численность населения приводится как итоговое состояние на конкретную дату (1 января), а не среднегодовое значение.

В исходной Excel-таблице данные организованы сложным образом, а **итоговая таблица** после приведения данных к единому формату имеет вид:

| Признаки       | Описание                                             |
| -------------- | ---------------------------------------------------- |
| **region**     | субъект Российской Федерации                         |
| **year**       | год наблюдения (дата 1 января соответствующего года) |
| **population** | численность постоянного населения региона, человек   |

Период, используемый в проекте: 2015-2020 гг.

Численность постоянно проживающего населения - ключевой демографический параметр региона. Без него невозможно корректно сопоставлять другие показатели.

In [9]:
pop_path = '../data_raw/population.xlsx'  
population = pd.read_excel(pop_path, sheet_name='Отчет')

# Формируем имена колонок по нулевой строке
header_row = population.iloc[0].tolist()

new_cols = []

for j, val in enumerate(header_row):
    if j == 0:
        new_cols.append('region')   # вместо NaN
    elif j == 1:
        new_cols.append('code')     # вместо NaN
    else:
        text = str(val)
        # Ищем год 
        match = re.search(r'(\d{4})', text)
        if match:
            new_cols.append(int(match.group(1))) 

# Присваиваем новые имена колонкам
population.columns = new_cols

# Убираем первые две строки (неинформативны) 
population = population.iloc[2:].reset_index(drop=True)

# Создаём вспомогательное поле с регионом
population['region_subject'] = population['region'].astype(str).str.strip()

# Значения "Раздел 1..." не являются названием региона, их временно очищаем
mask_div = population['region_subject'].str.startswith(
    'Раздел 1. Муниципальные образования субъектов Российской Федерации',
    na=False
)
population.loc[mask_div, 'region_subject'] = pd.NA

# Протягиваем последнее нормальное название региона вниз
population['region_subject'] = population['region_subject'].ffill()

# Оставляем строки, где есть итог по региону (код 00000000000 и уже подтянутый region_subject)
population['code'] = population['code'].astype(str).str.strip()
population = population.loc[(population['code'] == '00000000000')&(population['region_subject'].notna())].copy()

# Оставляем только регион и годы
year_cols = [col for col in population.columns if isinstance(col, int)]
population = population[['region_subject'] + year_cols].copy()

# Переименовываем регион в единый столбец 'region'
population = population.rename(columns={'region_subject': 'region'})

# Приводим таблицу к нужному виду
population = wide_years_to_long(population,
                                id_vars=['region'],
                                value_name='population',
                                year_col='year'
                               )

# Убираем строки без населения
population = population.dropna(subset=['population']).copy()

# Чистим регионы
population = clean_regions(population, region_col='region')

# Фильтруем годы 
population = filter_years(population, year_col='year')

# Финализируем таблицу
population = population[['region', 'year', 'population']].reset_index(drop=True)

print('Годы в population:', sorted(population['year'].unique()))
display(population.head())

Годы в population: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,population
0,Белгородская область,2015,1547936.0
1,Брянская область,2015,1232940.0
2,Владимирская область,2015,1405613.0
3,Воронежская область,2015,2331147.0
4,Ивановская область,2015,1036909.0


#### 3. Инвалидность по возрастным группам.

Число людей с инвалидностью по регионам, по месяцам, по возрастным группам.

**Файл**: disabled_total_by_age_2017_2022.csv

**Единица измерения**: человек.

Описание **исходных данных**: таблица содержит ежемесячные данные о численности населения с установленной группой инвалидности по субъектам Российской Федерации, с разбивкой по возрастным группам.

Поскольку данные представлены помесячно, для получения сопоставимых годовых значений в проекте берём данные за последний доступный месяц каждого года (фактически - данные на конец года).

После очистки и преобразования **итоговая таблица** имеет вид:

| Признаки             | Описание                                   |
| -------------------- | ------------------------------------------ |
| **region**           | субъект Российской Федерации               |
| **year**             | год наблюдения                             |
| **disabled_total**   | общее число инвалидов в регионе            |
| **disabled_18_30**   | численность инвалидов в возрасте 18-30 лет |
| **disabled_31_40**   | численность инвалидов в возрасте 31-40 лет |
| **disabled_41_50**   | численность инвалидов в возрасте 41-50 лет |
| **disabled_51_60**   | численность инвалидов в возрасте 51-60 лет |
| **disabled_60_plus** | численность инвалидов в возрасте 60+       |

Период: 2017-2020 годы.

Инвалидность - один из наиболее информативных индикаторов здоровья населения, качества медицинской помощи, социальной защищённости и условий жизни в регионе. Высокая доля инвалидов среди людей до пенсионного возраста снижает экономический потенциал региона и является прямым индикатором социального неблагополучия.

In [10]:
disabled_path = '../data_raw/disabled_total_by_age_2017_2022.csv'

disabled = pd.read_csv(disabled_path)

# Из даты берем год и месяц
disabled['date'] = pd.to_datetime(disabled['date'])
disabled['year'] = disabled['date'].dt.year
disabled['month'] = disabled['date'].dt.month

# Берём последнюю доступную запись в году
disabled = (disabled
            .sort_values(['region', 'year', 'month'])
            .drop_duplicates(subset=['region', 'year'], keep='last')
            .reset_index(drop=True)
           )

# Переименовываем признаки
disabled = disabled.rename(columns={'total': 'disabled_total',
                                    '18_30': 'disabled_18_30',
                                    '31_40': 'disabled_31_40',
                                    '41_50': 'disabled_41_50',
                                    '51_60': 'disabled_51_60',
                                    '60_': 'disabled_60_plus'
                                   })

# Чистим регионы
disabled = clean_regions(disabled, region_col='region')

# Фильтруем период
disabled = filter_years(disabled, year_col='year')

# Оставляем нужные колонки и упорядочиваем
disabled = disabled[['region',
                     'year',
                     'disabled_total',
                     'disabled_18_30',
                     'disabled_31_40',
                     'disabled_41_50',
                     'disabled_51_60',
                     'disabled_60_plus']].reset_index(drop=True)


print('Годы в disabled:', sorted(disabled['year'].unique()))
display(disabled.head())

Годы в disabled: [np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020)]


,region,year,disabled_total,disabled_18_30,disabled_31_40,disabled_41_50,disabled_51_60,disabled_60_plus
0,Алтайский край,2017,178967.0,8273.0,13553.0,14253.0,28820.0,114068.0
1,Алтайский край,2018,175193.0,7654.0,13491.0,14884.0,26850.0,112314.0
2,Алтайский край,2019,173868.0,7243.0,13482.0,15477.0,25395.0,112271.0
3,Алтайский край,2020,168269.0,7136.0,13409.0,16028.0,23552.0,108144.0
4,Амурская область,2017,67214.0,3438.0,5301.0,6275.0,11829.0,40371.0


#### 4. Рождаемость (число родившихся).

Рождённые по регионам, без учёта мертворождённых, помесячная демографическая статистика.

**Файл**: newborn_2006_2022_monthly.csv

**Единица измерения**: человек.

Описание **исходных данных**: в исходной таблице представлены месячные данные о числе родившихся по каждому субъекту Российской Федерации. Для целей проекта необходимо получить годовые значения, поэтому для каждого региона данные агрегируются суммированием всех месяцев года.

После очистки и преобразования **итоговая таблица** имеет вид:

| Признаки      | Описание                                      |
| ------------ | --------------------------------------------- |
| **region**   | субъект Российской Федерации                  |
| **year**     | год наблюдения (по данным за все месяцы года) |
| **newborns** | число родившихся за год, человек              |

Период, используемый в проекте: 2015-2020 гг.

Рождаемость - центральный показатель демографического благополучия региона. Снижение рождаемости - один из ключевых индикаторов демографической депрессии.

In [11]:
newborn_path = '../data_raw/newborn_2006_2022_monthly.csv'

newborn = pd.read_csv(newborn_path, sep=';')

# Удаляем неинформативные столбцы (Unnamed)
newborn = newborn.loc[:, ~newborn.columns.str.contains('^Unnamed')]

# Переименовываем колонку с регионом и чистим
newborn = newborn.rename(columns={'Region': 'region'})
newborn = clean_regions(newborn, region_col='region')

# Столбцы-месяцы 
month_cols = [col for col in newborn.columns if col != 'region']

newborn = newborn.melt(id_vars='region',
                       value_vars=month_cols,
                       var_name='month_str',
                       value_name='newborns'
                      )

# Извлекаем год из строки с месяцем
def extract_year(text):
    match = re.search(r'(\d{4})', str(text))
    return int(match.group(1)) if match else None

newborn['year'] = newborn['month_str'].apply(extract_year)

# Значения в числа
newborn['newborns'] = pd.to_numeric(newborn['newborns'], errors='coerce')

# Суммируем значения по каждому году и региону
newborn = (newborn.groupby(['region', 'year'], as_index=False).agg(newborns=('newborns', lambda x: x.sum(min_count=1))))

# Фильтруем период
newborn = filter_years(newborn, year_col='year')

# Переупорядочиваем колонки
newborn = newborn[['region', 'year', 'newborns']].reset_index(drop=True)

print('Годы в newborn:', sorted(newborn['year'].unique()))
display(newborn.tail(5))

Годы в newborn: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,newborns
565,г. Севастополь,2016,5409.0
566,г. Севастополь,2017,4845.0
567,г. Севастополь,2018,4366.0
568,г. Севастополь,2019,4267.0
569,г. Севастополь,2020,4225.0


#### 5. Детская смертность (город / село).

Число умерших на первом году жизни детей за год, по всем регионам, в городской и сельской местности.

**Файлы**:

- child_mortality_urban_1990_2021.xls - городское население;
- child_mortality_rural_1990_2021.xls - сельское население.

**Единица измерения**: человек (абсолютное число умерших детей в возрасте до 1 года).

Описание **исходных данных**: исходные таблицы содержат официальные данные Росстата о числе умерших детей на первом году жизни по субъектам Российской Федерации, с разбивкой:
- отдельно для городского населения;
- отдельно для сельского населения.

Показатель отражает годовой итог по каждому региону.

**Итоговая таблица**: 

| Признаки               | Описание                                                                     |
| ---------------------- | ---------------------------------------------------------------------------- |
| **region**             | субъект Российской Федерации                                                 |
| **year**               | год наблюдения                                                               |
| **child_deaths_urban** | число умерших детей в возрасте до 1 года среди городского населения, человек |
| **child_deaths_rural** | число умерших детей в возрасте до 1 года среди сельского населения, человек  |

Период: 2015-2020 гг.

Детская смертность - один из наиболее чувствительных индикаторов качества жизни и медицинского обслуживания в регионе. 
Разделение на город и село особенно важно, потому что:
- сельские территории обычно имеют худший доступ к высокотехнологичной медицине,
- различается структура рождаемости и социальные условия,
- различия между городской и сельской детской смертностью позволяют выявить внутреннее неравенство внутри региона.

In [12]:
cmu_path = '../data_raw/child_mortality_urban_1990_2021.xls'
cmr_path = '../data_raw/child_mortality_rural_1990_2021.xls'

cmu = load_excel_region_years(path=cmu_path,
                              sheet_name=0,
                              header_row=2, # шапка на 3-й строке
                              region_col_index=0, # первый столбец - регион
                              value_name="child_deaths_urban"
                             )

cmr = load_excel_region_years(path=cmr_path,
                              sheet_name=0,
                              header_row=2,
                              region_col_index=0,
                              value_name="child_deaths_rural"
                             )

# Объединим обе таблицы в одну
child_mortality = cmu.merge(cmr,
                            on=["region", "year"],
                            how="outer"
                           ).reset_index(drop=True)

child_mortality = child_mortality[["region", "year", "child_deaths_urban", "child_deaths_rural"]]

print("Годы в child_mortality:", sorted(child_mortality["year"].unique()))
display(child_mortality.tail(5))

Годы в child_mortality: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,child_deaths_urban,child_deaths_rural
625,г. Севастополь,2016,24.0,2.0
626,г. Севастополь,2017,19.0,1.0
627,г. Севастополь,2018,12.0,1.0
628,г. Севастополь,2019,15.0,0.0
629,г. Севастополь,2020,15.0,0.0


### 6. Бедность по социально-демографическим группам населения.

Распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017-2020 гг., по регионам.

**Файлы**: poverty_socdem_2017.xls, poverty_socdem_2018.xls, poverty_socdem_2019.xls, poverty_socdem_2020.xls

**Единица измерения**: процент (% малоимущего населения в соответствующей группе).

Описание **исходных данных**: в таблицах poverty_socdem_20**.xls по каждому региону представлена доля малоимущих (населения с доходами ниже величины прожиточного минимума) в разрезе основных социально-демографических групп:
- дети в возрасте до 16 лет;
- население старше трудоспособного возраста (пенсионеры);
- население трудоспособного возраста.

По тексту описания:

Малоимущее население - население с уровнем денежных доходов ниже величины прожиточного минимума.
Величина прожиточного минимума представляет собой расчётную величину, исходя из фактического состава обследуемого домохозяйства и величины прожиточного минимума по социально-демографическим группам населения, установленной в субъекте Российской Федерации по месту проживания обследуемого домохозяйства.

После обработки **итоговая таблица** имеет вид:

| Признаки                | Описание                                                                                                          |
| ----------------------- | ----------------------------------------------------------------------------------------------------------------- |
| **region**              | субъект Российской Федерации                                                                                      |
| **year**                | год наблюдения                                                                                                    |
| **poor_children_share** | доля детей в возрасте до 16 лет в общей численности малоимущего населения региона, %                              |
| **poor_old_share**      | доля населения старше трудоспособного возраста (пенсионеров) в общей численности малоимущего населения региона, % |
| **poor_working_share**  | доля населения трудоспособного возраста в общей численности малоимущего населения региона, %                      |

Все показатели выражены в процентах от общей численности малоимущего населения региона.

Период: 2017-2020 гг.

Эта таблица позволяет проанализировать структуру бедности:
- в каких регионах особенно велика доля детей среди малоимущих,
- где значительная часть бедного населения представлена пенсионерами,
- в каких регионах среди малоимущих преобладают люди трудоспособного возраста.

In [13]:
def load_socdem_year(year: int):
    """
    Загружает файл poverty_socdem_<year>.xls (лист 'Данные')
    и приводит к виду:
    region | year | poor_children_share | poor_old_share | poor_working_share
    """
    path = f"../data_raw/poverty_socdem_{year}.xls"
    
    # Читаем лист "Данные", шапка на 3-й строке (index=2)
    df = pd.read_excel(path,
                       sheet_name="Данные",
                       header=2
                      )
    
    # Первая колонка - регионы
    first_col = df.columns[0]
    df = df.rename(columns={first_col: "region"})
    
    # Переименовываем остальные колонки
    df = df.rename(columns={"Дети в возрасте до 16 лет": "poor_children_share",
                            "Население старше трудоспособного возраста": "poor_old_share",
                            "Население трудоспособного возраста": "poor_working_share"
                           })
    
    # Удаляем строки, где по всем показателям NaN 
    df = df.dropna(subset=["poor_children_share", "poor_old_share", "poor_working_share"],
                   how="all"
                  ).copy()
    
    # Добавляем год
    df["year"] = year
    
    # Чистим регионы
    df = clean_regions(df, region_col="region")

    # Исправляем названия регионов
    df["region"] = df["region"].replace({
        "Архангельская область (без АО)": 
            "Архангельская область (кроме Ненецкого автономного округа)",
        "Тюменская область (без АО)": 
            "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)"
    })

    # Упорядочиваем колонки
    df = df[["region",
             "year",
             "poor_children_share",
             "poor_old_share",
             "poor_working_share"
            ]].reset_index(drop=True)
    
    return df

In [14]:
years = [2017, 2018, 2019, 2020]
socdem_list = []

for y in years:
    df_y = load_socdem_year(y)
    socdem_list.append(df_y)

socdem = pd.concat(socdem_list, ignore_index=True)

print("Годы в socdem:", sorted(socdem["year"].unique()))
display(socdem.head())

Годы в socdem: [np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,poor_children_share,poor_old_share,poor_working_share
0,Белгородская область,2017,43.4,11.8,44.8
1,Брянская область,2017,42.9,4.9,52.2
2,Владимирская область,2017,34.8,8.6,56.6
3,Воронежская область,2017,38.6,5.9,55.6
4,Ивановская область,2017,33.2,10.9,55.9


#### 7. Жилищные условия домохозяйств.

**Файл**: housing_2020.xlsx

Файл содержит два листа:
- housing_cond - «Характеристика жилищных условий домохозяйств, % от всех домохозяйств»
- housing_intent - «Оценка домохозяйствами состояния занимаемого ими жилого помещения, % от всех домохозяйств»

**Единица измерения**: процент (% от всех домохозяйств региона), несколько переменных - в кв. метрах или количестве комнат.

Описание **исходных данных**: первый лист содержит данные о базовых жилищных условиях домохозяйств:
- насколько домохозяйства испытывают стеснённость в проживании,
- сколько у них жилой и общей площади,
- сколько комнат приходится на домохозяйство.

Эта таблица отражает качество жилищной обеспеченности, что является важным социальным компонентом благополучия.

Второй лист характеризует: 
- насколько домохозяйства удовлетворены состоянием своего жилья,
- планируют ли они улучшать жилищные условия,
- по каким причинам хотят улучшать,
- какие действия намерены предпринять.

Эта таблица показывает субъективную оценку комфортности жилья и жилищные стратегии, что важно для анализа социально-экономического положения регионов.

**Итоговая таблица** содержит структурированную совокупность всех жилищных индикаторов.
| Признаки                       | Описание                                   |
| ------------------------------ | ------------------------------------------ |
| **region**                     | субъект Российской Федерации               |
| **year**                       | год наблюдения (в проекте: 2020)           |
| **hh_no_crowding**             | доля проживающих без стеснённости          |
| **hh_some_crowding**           | доля проживающих с умеренной стеснённостью |
| **hh_high_crowding**           | доля проживающих в сильной стеснённости    |
| **area_total_per_capita**      | общая площадь на 1 человека, кв. м         |
| **area_living_per_capita**     | жилая площадь на 1 человека, кв. м         |
| **rooms_per_hh**               | число комнат на 1 домохозяйство            |
| **hh_plan_improve**            | планируют улучшить жилищные условия        |

Данные за 2020 год.

Жилищные условия - ключевой показатель качества жизни.
Они отражают:
- уровень обеспеченности жильём,
- степень перенаселённости квартир,
- субъективную оценку состояния жилья,
- жилищные стратегии домохозяйств,
- доступность покупок и улучшений жилья.

In [15]:
def process_housing_sheet(path: str,
                          sheet_name: str,
                          rename_map: dict,
                          numeric_cols: list,
                          year: int = 2020
                         ):
    """
    Загружает один лист из housing_2020.xlsx и приводит его к виду:
        region | year | <признаки...>    
    """

    df = pd.read_excel(path, sheet_name=sheet_name)

    # Первый столбец region
    df = df.rename(columns={df.columns[0]: "region"})

    # "..." в NaN
    df = df.replace("...", np.nan, regex=False)
    
    # Удаляем строки, где по всем признакам пусто
    df = df.dropna(subset=df.columns[1:], how="all")
    
    # Чистим регионы
    df = clean_regions(df, region_col="region")

    # Переименовываем признаки
    df = df.rename(columns=rename_map)

    # Числовые колонки приводим к числам
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Добавляем год
    df["year"] = year

    # Упорядочиваем: сначала region, year, потом остальные
    cols_order = ["region", "year"] + [c for c in df.columns if c not in ("region", "year")]
    df = df[cols_order]

    return df

In [16]:
h_path = "../data_raw/housing_2020.xlsx"

# --- housing_cond ---
cond_rename_map = {
    'Число домохозяйств, проживающих во всех типах жилых помещений': 'hh_total',
    'в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности': 'hh_no_crowding',
    'в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность': 'hh_some_crowding',
    'в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность': 'hh_high_crowding',
    'затруднились ответить': 'hh_crowding_dk',
    'Размер общей площади в расчете на члена домохозяйства': 'area_total_per_capita',
    'Размер жилой площади в расчете на члена домохозяйства': 'area_living_per_capita',
    'Число жилых комнат в расчете на одно домохозяйство': 'rooms_per_hh'
}

cond_numeric_cols = ['hh_total',
                     'hh_no_crowding',
                     'hh_some_crowding',
                     'hh_high_crowding',
                     'hh_crowding_dk',
                     'area_total_per_capita',
                     'area_living_per_capita',
                     'rooms_per_hh'
                    ]

housing_cond = process_housing_sheet(path=h_path,
                                     sheet_name="housing_cond",
                                     rename_map=cond_rename_map,
                                     numeric_cols=cond_numeric_cols,
                                     year=2020
                                    )

# --- housing_intent ---
intent_rename_map = {
    'из них домохозяйства, собирающиеся улучшить свои жилищные условия': 'hh_plan_improve'
}

intent_numeric_cols = ['hh_plan_improve']

housing_intent = process_housing_sheet(path=h_path,
                                       sheet_name="housing_intent",
                                       rename_map=intent_rename_map,
                                       numeric_cols=intent_numeric_cols,
                                       year=2020
                                      )

# Склеиваем оба листа в одну таблицу housing
housing = housing_cond.merge(housing_intent,
                             on=["region", "year"],
                             how="outer"
                            )

# Оставляем только нужные признаки
housing = housing[["region",
                   "year",
                   "hh_no_crowding",
                   "hh_some_crowding",
                   "hh_high_crowding",
                   "area_total_per_capita",
                   "area_living_per_capita",
                   "rooms_per_hh",
                   "hh_plan_improve"
                  ]].reset_index(drop=True)

print("Годы в housing:", housing["year"].unique())
display(housing.head())

Годы в housing: [2020]


,region,year,hh_no_crowding,hh_some_crowding,hh_high_crowding,area_total_per_capita,area_living_per_capita,rooms_per_hh,hh_plan_improve
0,Алтайский край,2020,83.2,13.4,3.3,25.9,18.0,2.4,14.9
1,Амурская область,2020,77.3,16.8,5.9,23.2,17.3,2.3,14.7
2,Архангельская область (кроме Ненецкого автоном...,2020,83.3,14.5,2.2,24.6,16.9,2.4,13.4
3,Астраханская область,2020,77.5,19.4,3.1,24.9,19.2,2.3,16.4
4,Белгородская область,2020,86.3,11.7,2.0,25.1,17.8,2.6,12.0


#### 8. Cведения о заболеваемости алкоголизмом и наркоманией.

**Файл**: drug_alco.xls

**Единица измерения**: человек на 100 000 населения.

Описание **исходных данных**: таблица содержит сведения о заболеваемости алкоголизмом и наркоманией по субъектам Российской Федерации. Показатель рассчитывается как производное числа зарегистрированных больных с впервые в жизни установленным диагнозом алкоголизма и алкогольного психоза, умноженного на 100 000 и делённого на численность населения на конец года.

Аналогично рассчитывается показатель по наркомании, и в таблицу включён сводный показатель, отражающий уровень заболеваемости, связанной с употреблением алкоголя и наркотиков.

Таким образом, в значении показателя drug_alco_rate учитывается:
- число новых случаев (впервые установленный диагноз),
- обычный годовой расчётный интервал (за год),
- нормировка на 100 000 человек постоянного населения региона.

**Итоговая таблица**:

| Признаки           | Описание                                                                                                     |
| ------------------ | ------------------------------------------------------------------------------------------------------------ |
| **region**         | субъект Российской Федерации                                                                                 |
| **year**           | год наблюдения                                                                                               |
| **drug_alco_rate** | число впервые зарегистрированных случаев алкоголизма/алкогольных психозов и наркоманий, на 100 000 населения |

Период: 2017-2020 гг.

Заболеваемость алкоголизмом и наркоманией - один из наиболее острых индикаторов социального неблагополучия и рискового поведения населения. Этот показатель отражает:
- распространённость злоупотребления алкоголем и психоактивными веществами,
- качество и доступность профилактической медицины,
- эффективность антиалкогольной и антинаркотической политики региона,
- уровень социальной напряжённости, маргинализации и криминогенной среды.

Важно, что используется именно первичная заболеваемость (впервые установленные диагнозы), а не распространённость. Это позволяет:
- отслеживать динамику новых случаев во времени,
- анализировать всплески и спады, связанные с кризисами, изменениями законодательства, кампаний и т.д.

In [17]:
drug_path = "../data_raw/drug_alco.xls"

drug = load_excel_region_years(path=drug_path,
                               sheet_name=0,
                               header_row=2, # 3-я строка - шапка с годами
                               region_col_index=0, # первый столбец - регион
                               value_name="drug_alco_rate"
                              )

print("Годы в drug:", sorted(drug["year"].unique()))
display(drug.head())

Годы в drug: [np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,drug_alco_rate
0,Белгородская область,2018,41.3
1,Брянская область,2018,82.2
2,Владимирская область,2018,69.9
3,Воронежская область,2018,83.6
4,Ивановская область,2018,81.6


#### 9. Доля занятых среди трудоспособного населения.

Отношение числа занятых в экономике региона к численности населения региона в трудоспособном возрасте  (процент).

**Файл**: workers.xls

**Единица измерения**: процент (%).

Описание **исходных данных**: показатель отражает долю занятых в экономике среди населения трудоспособного возраста региона. 

$$
\text{Доля занятых} =
\frac{\text{Численность занятых в экономике}}
{\text{Численность населения трудоспособного возраста}}
\times 100\%
$$

Этот показатель используется для оценки экономической активности населения и характеризует состояние рынка труда.
Для расчёта показателя, начиная с итогов за 2013 год, использована среднегодовая численность занятых в экономике, сформированная в соответствии с актуализированной методикой расчёта баланса трудовых ресурсов и оценки затрат труда.

**Итоговая таблица**:

| Признаки          | Описание                                                 |
| ----------------- | -------------------------------------------------------- |
| **region**        | субъект Российской Федерации                             |
| **year**          | год наблюдения                                           |
| **workers_share** | Отношение числа занятых в экономике региона к численности населения региона в трудоспособном возрасте, % |


Период: 2015-2020 годы.

Доля занятых среди трудоспособного населения - ключевой индикатор экономической активности региона. Он отражает:
- способность экономики региона создавать рабочие места,
- уровень официальной занятости,
- развитие промышленности и сферы услуг,
- состояние регионального рынка труда,
- устойчивость экономики к кризисам.

Низкие значения workers_share указывают:
- на высокую долю иждивенцев,
- депопуляцию трудоспособного населения,
- слабую экономику и ограниченный рынок труда,
- вероятное сочетание с алкоголизацией, инвалидностью и низкими доходами.

In [18]:
workers_path = "../data_raw/workers.xls"

workers = load_excel_region_years(path=workers_path,
                                  sheet_name=0,
                                  header_row=2, # шапка на 3-й строке
                                  region_col_index=1, # ВТОРАЯ колонка - region (первая неинформативная)
                                  value_name="workers_share"
                                 )

print("Годы в workers:", sorted(workers["year"].unique()))
display(workers.head())

Годы в workers: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,workers_share
0,Белгородская область,2015,84.8
1,Брянская область,2015,78.3
2,Владимирская область,2015,85.2
3,Воронежская область,2015,82.0
4,Ивановская область,2015,78.0


#### 10. Доходы населения и заработная плата по регионам

**Файл**: cash_real_income_wages_2015_2020.xlsx

**Единицы измерения**:
- Рубли в месяц (среднедушевые денежные доходы, номинальная заработная плата)
- Проценты к предыдущему году (реальные доходы, реальная зарплата)

Описание **исходных данных**: файл содержит ключевые показатели, характеризующие уровень материального благополучия населения и динамику доходов. Все показатели рассчитаны в соответствии с действующей методологией и представлены ежегодно по субъектам Российской Федерации.

В составе файла 4 таблицы: 
- Среднедушевые денежные доходы населения - это показатель фактических денежных поступлений в расчёте на одного жителя.
- Реальные денежные доходы населения - этот показатель показывает выросли ли доходы людей с учётом изменения цен.
- Среднемесячная номинальная начисленная заработная плата - это заработная плата работников организаций, начисленная до удержания налогов.
- Реальная начисленная заработная плата - показывает, улучшилось ли благосостояние работающего населения фактически, а не только номинально.

**Итоговая таблица** показателей доходов и заработной платы:

| Признаки                   | Описание                                                      |
| -------------------------- | ------------------------------------------------------------- |
| **region**                 | субъект Российской Федерации                                  |
| **year**                   | год наблюдения                                                |
| **cash_income_per_capita** | среднедушевые денежные доходы населения, руб./мес             |
| **real_incomes_index**     | реальные денежные доходы населения, % к предыдущему году      |
| **formal_wage_paid**       | среднемесячная номинальная начисленная заработная плата, руб. |
| **real_wage_index**        | реальная заработная плата, % к предыдущему году               |

Период: 2015-2020 годы.

Данные о доходах и заработной плате - центральный элемент оценки экономического благополучия населения.

In [19]:
cash_path = '../data_raw/cash_real_income_wages_2015_2020.xlsx'

# Лист Excel = имя признака в итоговой таблице
cash_sheet_value_map = {
    'per_capita_cash_income': 'cash_income_per_capita',  # среднедушевые денежные доходы
    'real_incomes': 'real_incomes_index',                # индекс реальных доходов
    'formal_wage_paid': 'formal_wage_paid',              # номинальная начисленная зарплата
    'real_pay': 'real_wage_index',                       # индекс реальной начисленной зарплаты
}

cash = None

for i, (sheet_name, value_name) in enumerate(cash_sheet_value_map.items()):
    cash_part = load_excel_years(path=cash_path,
                                 value_name=value_name,
                                 sheet_name=sheet_name,
                                 header=0         # шапка в первой строке
                                 )
    if i == 0:
        cash = cash_part
    else:
        cash = cash.merge(cash_part,
                          on=['region', 'year'],
                          how='outer'
                         )

print('Годы в cash:', sorted(cash['year'].unique()))
display(cash.head())

Годы в cash: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,cash_income_per_capita,real_incomes_index,formal_wage_paid,real_wage_index
0,Алтайский край,2015,20860.0,99.1,20090.0,90.0
1,Алтайский край,2016,21256.0,94.7,21202.0,98.4
2,Алтайский край,2017,22139.0,100.0,22743.0,103.6
3,Алтайский край,2018,22829.0,99.7,25519.0,109.3
4,Алтайский край,2019,23937.0,99.6,27962.0,104.9


#### 11. Расходы на социальную политику в структуре бюджета региона.

**Файл**: welfare_expense_share_2015_2020.xlsx

**Единицы измерения**: Процент (% от общего объёма расходов консолидированного бюджета субъекта РФ).

Описание **исходных данных**: для каждого региона и года рассчитывается доля того, какую часть своего бюджета регион тратит на социальную сферу.
$$
\text{welfare\_expense\_share} =
\frac{\text{Расходы на социальную политику}}
{\text{Общие расходы консолидированного бюджета региона}}
\times 100\%
$$

Под расходами на социальную политику обычно понимают (в рамках бюджетной классификации):
- социальную поддержку населения (пособия, льготы),
- поддержку семей с детьми,
- меры по поддержке инвалидов, пожилых, малоимущих,
- отдельные виды социальных выплат и программ.

Речь идёт о консолидированном бюджете субъекта Российской Федерации - то есть суммарном бюджете: регионального уровня + местных бюджетов  (муниципалитетов) в границах региона.

**Итоговая таблица**: 

| Признаки                 | Описание                                                                          |
| ------------------------- | --------------------------------------------------------------------------------- |
| **region**                | субъект Российской Федерации                                                      |
| **year**                  | год наблюдения                                                                    |
| **welfare_expense_share** | доля расходов на социальную политику в общем объёме бюджетных расходов региона, % |

Период: 2015-2020 годы.

Удельный вес расходов на социальную политику показывает приоритеты бюджетной политики региона:
- насколько регион ориентирован на поддержку населения,
- готов ли он перераспределять ресурсы в пользу более уязвимых групп,
- насколько значим социальный блок по отношению к другим статьям расходов (инфраструктура, управление, безопасность, экономика и т.п.).

In [20]:
welfare_path = '../data_raw/welfare_expense_share_2015_2020.xlsx'

welfare = load_excel_years(path=welfare_path,
                           value_name='welfare_expense_share',
                           sheet_name=0,
                           header=0       # шапка в первой строке
                          )

print('Годы в welfare:', sorted(welfare['year'].unique()))
display(welfare.head())

Годы в welfare: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,welfare_expense_share
0,Белгородская область,2015,11.3
1,Брянская область,2015,22.0
2,Владимирская область,2015,18.1
3,Воронежская область,2015,15.2
4,Ивановская область,2015,18.1


#### 12. Валовой региональный продукт на душу населения.

**Файл**: gross_regional_product_1996_2020.xls

**Единица измерения**: рубль (валовой региональный продукт в расчёте на одного жителя).

Описание **исходных данных**: показатель показывает, какой объём созданной в регионе добавленной стоимости (ВРП) приходится в среднем на одного жителя.

**Итоговая таблица**: 

| Признаки           | Описание                                                             |
| ------------------ | -------------------------------------------------------------------- |
| **region**         | субъект Российской Федерации                                         |
| **year**           | год наблюдения                                                       |
| **grp_per_capita** | валовой региональный продукт на душу населения, руб. в текущих ценах |

Период: 2015-2020 годы.

ВРП на душу населения - один из центральных макроэкономических индикаторов, который:
- отражает уровень экономического развития региона,
- позволяет сравнивать регионы разного масштаба (малые/крупные по населению),
- даёт представление о том, какой экономический продукт создаётся в среднем на одного жителя.

In [21]:
grp_path = "../data_raw/gross_regional_product_1996_2020.xls"

grp = load_excel_region_years(path=grp_path,
                              sheet_name=0,
                              header_row=2, # шапка на 3-й строке
                              region_col_index=0, # первый столбец - регион
                              value_name="grp_per_capita"
                             )

print("Годы в grp:", sorted(grp["year"].unique()))
display(grp.head())

Годы в grp: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,grp_per_capita
0,Белгородская область,2015,447619.7
1,Брянская область,2015,221080.0
2,Владимирская область,2015,262945.5
3,Воронежская область,2015,345566.8
4,Ивановская область,2015,174687.5


#### 13. Промышленное производство по видам экономической деятельности

**Файлы**:
- regional_production_2005_2016.csv
- regional_production_2017_2020.csv

**Единица измерения**: тысячи рублей (объём отгруженных товаров / выполненных работ и услуг за год).

Описание **исходных данных**: в файлах содержатся данные об объёме отгруженных товаров собственного производства и работ/услуг, выполненных собственными силами, по видам экономической деятельности.

Показатель приводится за год, по субъектам Российской Федерации, в текущих ценах, в тысячах рублей.

Файлы разделены по периодам:
- regional_production_2005_2016.csv - данные за 2005-2016 гг.;
- regional_production_2017_2020.csv - данные за 2017-2020 гг.

**Итоговая таблица**: 
| Признаки           | Описание                                                                                                   |
| ------------------ | ---------------------------------------------------------------------------------------------------------- |
| **region**         | субъект Российской Федерации                                                                               |
| **year**           | год наблюдения                                                                                             |
| **industry_total** | общий объём промышленного производства, тыс. руб.                                                          |
| **mining**         | объём отгруженной продукции добычи полезных ископаемых, тыс. руб.                                          |
| **manufacturing**  | объём отгруженной продукции обрабатывающих производств, тыс. руб.                                          |
| **energy**         | производство и распределение электроэнергии, газа, пара и кондиционирования воздуха, тыс. руб.             |
| **water_waste**    | водоснабжение, водоотведение, сбор и утилизация отходов, деятельность по ликвидации загрязнений, тыс. руб. |

Период: 2015-2020 годы.

Промышленное производство - ключевой индикатор экономической мощности региона. Показатель характеризует:
- размер и структуру экономики региона;
- развитие сырьевых и перерабатывающих отраслей;
- энергетическую инфраструктуру;
- сферу жилищно-коммунальных услуг (через блок water_waste).

In [22]:
rp_path_1 = "../data_raw/regional_production_2005_2016.csv"
rp_path_2 = "../data_raw/regional_production_2017_2020.csv"

rp1 = pd.read_csv(rp_path_1)
rp2 = pd.read_csv(rp_path_2)

# Чистим регионы
rp1 = clean_regions(rp1, region_col="region")
rp2 = clean_regions(rp2, region_col="region")

# Приведение таблиц к нужному формату 
rp1 = wide_years_to_long(rp1,
                         id_vars=["region", "production_field"],
                         value_name="production_value",
                         year_col="year"
                        )

rp2 = wide_years_to_long(rp2,
                         id_vars=["region", "production_field"],
                         value_name="production_value",
                         year_col="year"
                        )

# Склеиваем в одну таблицу
rp = pd.concat([rp1, rp2], ignore_index=True)

# Оставляем только 2015-2020 (под наш основной период)
rp = filter_years(rp, year_col='year').reset_index(drop=True)

# Изменение названий отраслей
sector_map = {
    "РАЗДЕЛ С ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ": "mining",
    "ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ": "mining",

    "РАЗДЕЛ D ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА": "manufacturing",
    "ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА": "manufacturing",

    "РАЗДЕЛ E ПРОИЗВОДСТВО И РАСПРЕДЕЛЕНИЕ ЭЛЕКТРОЭНЕРГИИ, ГАЗА И ВОДЫ": "energy",
    "ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА": "energy",

    "ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ": "water_waste",

    "Промышленное производство (промышленность)": "industry_total",
}

rp["sector"] = rp["production_field"].map(sector_map)

# Группируем по региону, году и отрасли
rp = (rp.groupby(["region", "year", "sector"], as_index=False)["production_value"].sum())

# Приводим к нужному виду
rp = (rp.pivot(index=["region", "year"], columns="sector", values="production_value").reset_index())
rp.columns.name = None

# Общий объём промышленного производства
sector_cols = [c for c in ["mining", "manufacturing", "energy", "water_waste"]
               if c in rp.columns]

rp["industry_total"] = rp["industry_total"].fillna(rp[sector_cols].sum(axis=1, skipna=True))

# Переупорядочиваем колонки 
rp = rp[['region', 'year', 'industry_total', 'mining', 'manufacturing', 'energy', 'water_waste']].reset_index(drop=True)

print("Годы в rp:", sorted(rp["year"].unique()))
display(rp.head())

Годы в rp: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,industry_total,mining,manufacturing,energy,water_waste
0,Агинский Бурятский округ (Забайкальский край),2015,0.000000e+00,0.0,0.000000e+00,0.00,NaN
1,Агинский Бурятский округ (Забайкальский край),2016,0.000000e+00,0.0,0.000000e+00,0.00,NaN
2,Алтайский край,2015,3.215263e+08,5519930.2,2.716510e+08,44355402.78,NaN
3,Алтайский край,2016,3.157819e+08,4852175.4,2.658828e+08,45046977.27,NaN
4,Алтайский край,2017,3.679882e+08,6143029.4,3.079699e+08,45806615.70,8068621.4


#### 14. Оборот розничной торговли на душу населения.

**Файл**: retail_turnover_per_capita_2000_2021.xls

**Единица измерения**: рубли (оборот розничной торговли в расчёте на одного жителя региона).

Описание **исходных данных**: показатель отражает, сколько рублей розничного товарооборота приходится в среднем на одного жителя региона за год.

**Итоговая таблица**:

| Признаки            | Описание                                                |
| ------------------- | ------------------------------------------------------- |
| **region**          | субъект Российской Федерации                            |
| **year**            | год наблюдения                                          |
| **retail_turnover** | оборот розничной торговли на душу населения, руб. в год |

Период: 2015-2020 годы.

Оборот розничной торговли на душу населения - важный индикатор платёжеспособного спроса и экономической активности населения:
- показывает, насколько высок потребительский спрос в регионе,
- отражает уровень развития торговли и услуг,
- косвенно связан с доходами, урбанизацией, миграцией, уровнем инфраструктуры.

In [23]:
rt_path = "../data_raw/retail_turnover_per_capita_2000_2021.xls"

retail = load_excel_region_years(path=rt_path,
                                 sheet_name=0,
                                 header_row=2, # шапка на 3-й строке
                                 region_col_index=0, # первый столбец - регион
                                 value_name="retail_turnover"
                                )

# Убираем служебную строку "значение показателя за год"
retail = retail[~retail["region"].str.contains("значение показателя за год", case=False, na=False)].reset_index(drop=True)

# Убираем строки "без АО" 
retail = retail[~retail['region'].str.contains('без АО', na=False)].reset_index(drop=True)

print('Годы в retail:', sorted(retail['year'].unique()))
display(retail.head())

Годы в retail: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,retail_turnover
0,Белгородская область,2015,178097.0
1,Брянская область,2015,177048.0
2,Владимирская область,2015,139408.0
3,Воронежская область,2015,198814.0
4,Ивановская область,2015,143349.0


#### 15. Преступность.
    
**Файлы**: 
- 4-EGS_Razdel_4_122016.xls 
- 4-EGS_Razdel_4_122017.xls
- 4-EGS_Razdel_4_122018.xls
- 4-EGS_Razdel_4_122019.xls
- 4-EGS_Razdel_4_122020.xls
  
**Единица измерения**: количество зарегистрированных преступлений.

Накопительные данные за весь календарный год, отражённые в декабрьской форме отчётности.

**Исходные данные**: для каждого региона представлены показатели по числу совершённых преступлений с разбивкой по различным социальным и криминологическим характеристикам правонарушителей:
- преступления, совершённые несовершеннолетними (или при их участии);
- преступления, совершённые лицами, ранее совершавшими преступления;
- преступления, совершённые ранее судимыми лицами;
- преступления, совершённые группой лиц;
- преступления, совершённые группой лиц по предварительному сговору;
- преступления, совершённые организованной группой;
- преступления, совершённые преступным сообществом (организацией);
- преступления, совершённые в состоянии алкогольного, наркотического или токсического опьянения.

Данные представлены отдельно для каждого года. Поскольку отчётная форма содержит накопительные данные за год в декабре, показатели отражают итоговое годовое значение.

**Итоговая таблица**: 

| Признаки                   |  Описание                                                                  | Годы      |
| -------------------------- | -------------------------------------------------------------------------- | --------- |
| **crime_juveniles**          | преступления, совершённые несовершеннолетними или при их участии           | 2016-2020 |
| **crime_repeat_offenders**   | преступления, совершённые лицами, ранее совершавшими преступления          | 2016-2020 |
| **crime_prev_convicted**     | преступления, совершённые ранее судимыми лицами                            | 2016-2020 |
| **crime_group**              | преступления, совершённые группой лиц                                      | 2016-2020 |
| **crime_group_prep**         | преступления, совершённые группой лиц по предварительному сговору          | 2016-2020 |
| **crime_organized_group**    | преступления, совершённые организованной группой                           | 2016-2020 |
| **crime_criminal_community** | преступления, совершённые преступным сообществом (преступной организацией) | 2016-2020 |
| **crime_intox_alcohol**      | преступления, совершённые в состоянии алкогольного опьянения               | 2016-2020 |
| **crime_intox_narcotic**     | преступления, совершённые в состоянии наркотического опьянения             | 2016-2020 |
| **crime_intox_toxic**        | преступления, совершённые в состоянии токсического опьянения               | 2016-2020 |

Период: 2016-2020 годы.

Показатели преступности с разбивкой по социальным и криминологическим характеристикам правонарушителей позволяют:
- оценивать уровень социального неблагополучия региона;
- выявлять криминологические риски (молодёжная преступность, рецидив, организованность преступлений);
- анализировать связь между социально-экономическими условиями и уровнем преступности;
- использовать данные как факторы при кластеризации регионов;
- выявлять территории, где возможны проблемы социальной напряжённости.

In [24]:
base_path = "../data_raw/4-EGS_Razdel_4_"

frames = []

for year in range(2016, 2021):   # 2016-2020
    filename = f"12{year}.xls"   # 122016, 122017, ...
    path = base_path + filename

    df_year = load_crime_year_file(path, year=year)
    frames.append(df_year)

crime = pd.concat(frames, ignore_index=True)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [25]:
# Список колонок с показателями преступности
crime_indicator_cols = ["crime_juveniles",
                        "crime_repeat_offenders",
                        "crime_prev_convicted",
                        "crime_group",
                        "crime_group_prep",
                        "crime_organized_group",
                        "crime_criminal_community",
                        "crime_intox_alcohol",
                        "crime_intox_narcotic",
                        "crime_intox_toxic"
                       ]

# Годовая агрегированная таблица
crime = (crime.groupby(["region", "year"], as_index=False)[crime_indicator_cols].sum())

print("Годы в crime:", sorted(crime["year"].unique()))
display(crime.head())

Годы в crime: [np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]


,region,year,crime_juveniles,crime_repeat_offenders,crime_prev_convicted,crime_group,crime_group_prep,crime_organized_group,crime_criminal_community,crime_intox_alcohol,crime_intox_narcotic,crime_intox_toxic
0,Алтайский край,2016,1310,19308,7688,69,1480,94,0,13875,241,0
1,Алтайский край,2017,1134,17755,7947,47,1348,145,5,11976,345,1
2,Алтайский край,2018,1081,17117,7919,51,1371,25,32,10864,99,3
3,Алтайский край,2019,961,16553,7820,60,1156,77,5,9873,53,1
4,Алтайский край,2020,887,15400,7278,64,901,87,2,9396,46,0


Соединим все таблицы в один большой датасет.

In [26]:
# список таблиц для объединения
tables = [poverty,
          population,
          disabled,
          newborn,
          child_mortality,
          socdem,
          housing,
          drug,
          workers,
          cash,
          welfare,
          grp,
          rp,
          retail,
          crime]

# первая таблица - база для объединения
data = tables[0].copy()

# последовательно присоединяем остальные
for t in tables[1:]:
    data = data.merge(t, on=["region", "year"], how="left")

# Упорядочим колонки: сначала region, year, потом остальные 
base_cols = ["region", "year"]
other_cols = [c for c in data.columns if c not in base_cols]
data = data[base_cols + other_cols].reset_index(drop=True)
   
print(data.shape)

(522, 47)


#### Дополнительные (производные) Признаки

Для повышения информативности итоговой таблицы и улучшения качества анализа и кластеризации регионов рассчитывается несколько производных показателей на основе уже имеющихся данных.

| Признаки | Описание | Формула |
|--------|----------|---------|
| **poor_abs** | Оценка абсолютного числа людей, живущих за чертой бедности, человек. | $$\text{poor\_abs} = \text{population} \times \frac{\text{poverty\_percent}}{100}$$ |
| **production_pc** | Промышленное производство на душу населения, тыс. руб. Характеризует экономическую «мощность» региона в расчёте на одного жителя. | $$\text{production\_pc} = \frac{\text{industry\_total}}{\text{population}}$$ |
| **retail_to_income** | Отношение оборота розничной торговли на душу населения к среднедушевым денежным доходам. Показывает, насколько высок фактический потребительский расход относительно доходов. | $$\text{retail\_to\_income} = \frac{\text{retail\_turnover}}{\text{cash\_income\_per\_capita}}$$ |
| **birth_rate_per_1000** | Коэффициент рождаемости: число родившихся на 1000 человек населения. | $$\text{birth\_rate\_per\_1000} = \frac{\text{newborns}}{\text{population}} \times 1000$$ |
| **infant_mortality_per_1000** | Младенческая смертность: число умерших на первом году жизни на 1000 рождений (город + село вместе). | $$\text{infant\_mortality\_per\_1000} = \frac{\text{child\_deaths\_urban} + \text{child\_deaths\_rural}}{\text{newborns}} \times 1000$$ |
| **disabled_per_1000** | Число инвалидов на 1000 человек населения. Позволяет сопоставлять регионы с разным масштабом. | $$\text{disabled\_per\_1000} = \frac{\text{disabled\_total}}{\text{population}} \times 1000$$ |

Каждый из этих показателей рассчитывается только в тех случаях, когда в данных присутствуют необходимые исходные переменные и ненулевая база для деления.

In [27]:
def add_derived_features(df: pd.DataFrame):
    """
    Добавляет производные признаки на основе имеющихся данных:
      - poor_abs
      - production_pc
      - retail_to_income
      - birth_rate_per_1000
      - infant_mortality_per_1000
      - disabled_per_1000
    """
    
    #  Абсолютное число бедных
    if {"population", "poverty_percent"}.issubset(df.columns):
        df["poor_abs"] = df["population"] * df["poverty_percent"] / 100

    # Промышленное производство на душу населения
    if {"industry_total", "population"}.issubset(df.columns):
        # industry_total в тыс. руб., population в людях - показатель "тыс. руб. на человека"
        df["production_pc"] = df["industry_total"] / df["population"]

    # Отношение оборота розницы к доходам
    if {"retail_turnover", "cash_income_per_capita"}.issubset(df.columns):
        df["retail_to_income"] = df["retail_turnover"] / df["cash_income_per_capita"]

    # Рождаемость на 1000 человек
    if {"newborns", "population"}.issubset(df.columns):
        mask = df["population"] > 0
        df.loc[mask, "birth_rate_per_1000"] = (
            df.loc[mask, "newborns"] / df.loc[mask, "population"] * 1000
        )

    # Младенческая смертность на 1000 рождений
    if {"child_deaths_urban", "child_deaths_rural", "newborns"}.issubset(df.columns):
        total_child_deaths = (
            df["child_deaths_urban"].fillna(0) + df["child_deaths_rural"].fillna(0)
        )
        mask = df["newborns"] > 0
        df.loc[mask, "infant_mortality_per_1000"] = (
            total_child_deaths[mask] / df.loc[mask, "newborns"] * 1000
        )

    # Инвалидность на 1000 человек населения
    if {"disabled_total", "population"}.issubset(df.columns):
        mask = df["population"] > 0
        df.loc[mask, "disabled_per_1000"] = (
            df.loc[mask, "disabled_total"] / df.loc[mask, "population"] * 1000
        )

    return df

In [28]:
data = add_derived_features(data)
data["region"] = data["region"].replace({
        "Архангельская область (кроме Ненецкого автономного округа)": 
            "Архангельская область без АО",
        "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)": 
            "Тюменская область без АО"})
data = data.sort_values(['region', 'year']).reset_index(drop=True)

#### Итоговый датасет: 

| Признаки                  | Описание                                                                             | Период     |
| -------------------------| ------------------------------------------------------------------------------------ | -----------|
| **region**                 | Субъект Российской Федерации                                                         |            |
| **year**                   | Год наблюдения                                                                       |            |
| **poverty_percent**        | Доля населения с денежными доходами ниже величины прожиточного минимума, %           | 2015-2020  |
| **poor_children_share** | Доля детей в возрасте до 16 лет в общей численности малоимущего населения региона, %                              | 2017-2020  |
| **poor_old_share**      | Доля населения старше трудоспособного возраста (пенсионеров) в общей численности малоимущего населения региона, % | 2017-2020  |
| **poor_working_share**  | Доля населения трудоспособного возраста в общей численности малоимущего населения региона, %                      | 2017-2020  |
| **cash_income_per_capita** | Среднедушевые денежные доходы населения, руб./месяц                                  | 2015-2020  |
| **real_incomes_index**     | Индекс реальных денежных доходов населения, % к предыдущему году                     | 2015-2020  |
| **formal_wage_paid**       | Среднемесячная номинальная начисленная заработная плата работников организаций, руб. | 2015-2020  |
| **real_wage_index**        | Индекс реальной начисленной заработной платы, % к предыдущему году                   | 2015-2020  |
| **population**             | Численность постоянного населения региона, человек                                   | 2015-2020  |
| **newborns**               | Число родившихся за год, человек                                                     | 2015-2020  |
| **child_deaths_urban**     | Число умерших на первом году жизни среди городского населения, человек               | 2015-2020  |
| **child_deaths_rural**     | Число умерших на первом году жизни среди сельского населения, человек                | 2015-2020  |
| **disabled_total**         | Общее число инвалидов в регионе                                                      | 2017-2020  |
| **disabled_18_30**         | Число инвалидов в возрасте 18-30 лет                                                 | 2017-2020  |
| **disabled_31_40**         | Число инвалидов в возрасте 31-40 лет                                                 | 2017-2020  |
| **disabled_41_50**         | Число инвалидов в возрасте 41-50 лет                                                 | 2017-2020  |
| **disabled_51_60**         | Число инвалидов в возрасте 51-60 лет                                                 | 2017-2020  |
| **disabled_60_plus**       | Число инвалидов в возрасте 60 лет и старше                                           | 2017-2020  |
| **hh_no_crowding**             | Домохозяйства, не испытывающие стеснённости при проживании       | 2020          |
| **hh_some_crowding**           | Домохозяйства, испытывающие определённую стеснённость            | 2020          |
| **hh_high_crowding**           | Домохозяйства, испытывающие большую стеснённость                 | 2020          |
| **area_total_per_capita**      | Общая площадь жилья в расчёте на 1 члена домохозяйства, кв. м    | 2020          |
| **area_living_per_capita**     | Жилая площадь в расчёте на 1 члена домохозяйства, кв. м          | 2020          |
| **rooms_per_hh**               | Среднее число жилых комнат на одно домохозяйство                 | 2020          |
| **hh_plan_improve**            | Домохозяйства, собирающиеся улучшить жилищные условия            | 2020          |
| **drug_alco_rate** | Число впервые зарегистрированных случаев алкоголизма/алкогольных психозов и наркоманий, на 100 000 населения | 2018-2020 |
| **workers_share** | Отношение числа занятых в экономике региона к численности населения региона в трудоспособном возрасте, % | 2015-2020  |
| **welfare_expense_share** | Доля расходов на социальную политику в общем объёме расходов консолидированного бюджета региона, % | 2015-2020 |
| **grp_per_capita** | Валовой региональный продукт на душу населения, руб. | 2015-2020 |
| **industry_total** | Общий объём промышленного производства, тыс. руб. | 2015-2020 |
| **mining** | Объём производства в добыче полезных ископаемых, тыс. руб. | 2015-2020 |
| **manufacturing** | Объём производства в обрабатывающих производствах, тыс. руб. | 2015-2020 |
| **energy** | Производство и распределение электроэнергии, газа, пара и кондиционирования воздуха, тыс. руб. | 2015-2020 |
| **water_waste** | Водоснабжение, водоотведение, сбор/утилизация отходов, ликвидация загрязнений, тыс. руб. | 2015-2020 |
| **retail_turnover** | Оборот розничной торговли на душу населения, руб. в год | 2015-2020 |             
| **crime_juveniles**          | Преступления, совершённые несовершеннолетними или при их участии           | 2016-2020 |
| **crime_repeat_offenders**   | Преступления, совершённые лицами, ранее совершавшими преступления          | 2016-2020 |
| **crime_prev_convicted**     | Преступления, совершённые ранее судимыми лицами                            | 2016-2020 |
| **crime_group**              | Преступления, совершённые группой лиц                                      | 2016-2020 |
| **crime_group_prep**         | Преступления, совершённые группой лиц по предварительному сговору          | 2016-2020 |
| **crime_organized_group**    | Преступления, совершённые организованной группой                           | 2016-2020 |
| **crime_criminal_community** | Преступления, совершённые преступным сообществом (преступной организацией) | 2016-2020 |
| **crime_intox_alcohol**      | Преступления, совершённые в состоянии алкогольного опьянения               | 2016-2020 |
| **crime_intox_narcotic**     | Преступления, совершённые в состоянии наркотического опьянения             | 2016-2020 |
| **crime_intox_toxic**        | Преступления, совершённые в состоянии токсического опьянения               | 2016-2020 |
| **poor_abs**                  | Абсолютное число людей, живущих за чертой бедности. Рассчитывается как произведение численности населения на долю населения с доходами ниже прожиточного минимума. Характеризует реальный масштаб бедности в регионе. | 2015-2020        |
| **production_pc**             | Промышленное производство на душу населения. Отражает экономическую мощность региона в расчёте на одного жителя. Чем выше значение - тем выше производственная эффективность.                                         | 2015-2020   |
| **retail_to_income**          | Отношение оборота розничной торговли на душу населения к среднедушевым денежным доходам. Показывает интенсивность фактического потребления относительно доходов населения.                                            | 2015-2020  |
| **birth_rate_per_1000**       | Коэффициент рождаемости: число родившихся живыми на 1000 человек населения. Позволяет корректно сравнивать регионы независимо от размеров населения.                                                                  | 2015-2020                                        |
| **infant_mortality_per_1000** | Младенческая смертность: число умерших детей до 1 года на 1000 рождений. Высокие значения указывают на проблемы здравоохранения и социального благополучия.                                                           | 2015-2020   |
| **disabled_per_1000**         | Число людей с инвалидностью на 1000 человек населения. Позволяет сравнивать регионы по относительному уровню инвалидности, а не по абсолютным числам.                                                                 | 2017-2020            |

In [29]:
display(data)

,region,year,poverty_percent,population,disabled_total,disabled_18_30,disabled_31_40,disabled_41_50,disabled_51_60,disabled_60_plus,...,crime_criminal_community,crime_intox_alcohol,crime_intox_narcotic,crime_intox_toxic,poor_abs,production_pc,retail_to_income,birth_rate_per_1000,infant_mortality_per_1000,disabled_per_1000
0,Алтайский край,2015,18.0,2384812.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,429266.160,134.822506,6.468121,11.520824,8.080073,NaN
1,Алтайский край,2016,17.8,2376774.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,13875.0,241.0,0.0,423065.772,132.861575,6.484945,11.075937,8.015195,NaN
2,Алтайский край,2017,17.5,2365680.0,178967.0,8273.0,13553.0,14253.0,28820.0,114068.0,...,5.0,11976.0,345.0,1.0,413994.000,155.552820,6.498622,8.967823,8.720245,75.651398
3,Алтайский край,2018,17.4,2350080.0,175193.0,7654.0,13491.0,14884.0,26850.0,112314.0,...,32.0,10864.0,99.0,3.0,408913.920,164.077977,6.590039,9.995830,7.492231,74.547675
4,Алтайский край,2019,17.6,2332813.0,173868.0,7243.0,13482.0,15477.0,25395.0,112271.0,...,5.0,9873.0,53.0,1.0,410575.088,179.679629,6.663909,9.052161,5.398494,74.531478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,г. Севастополь,2016,11.7,416263.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,677.0,50.0,1.0,48702.771,52.861614,5.548039,12.994189,4.806803,NaN
518,г. Севастополь,2017,10.9,428753.0,19622.0,790.0,1498.0,1997.0,3487.0,11850.0,...,21.0,524.0,50.0,0.0,46734.077,53.562112,5.199573,11.300212,4.127967,45.765277
519,г. Севастополь,2018,10.8,436670.0,19866.0,752.0,1497.0,2012.0,3415.0,12190.0,...,0.0,625.0,33.0,0.0,47160.360,81.286026,5.176354,9.998397,2.977554,45.494309
520,г. Севастополь,2019,11.6,443212.0,20330.0,770.0,1501.0,2074.0,3334.0,12651.0,...,0.0,511.0,13.0,0.0,51412.592,55.977302,5.113314,9.627447,3.515350,45.869697


Выгрузим итоговый датасет в отдельный файл.

In [30]:
data.to_csv("../data_processed/data_final.csv", index=False, sep=";", encoding="utf-8-sig")

### Вывод по первой части проекта

В данной части проекта была выполнена масштабная подготовка данных, включающая сбор, очистку, приведение к единому формату и объединение разнородных статистических показателей по регионам России. Работа охватывала более десяти независимых источников данных, различающихся по формату, структуре, единицам измерения и периодам наблюдения.

Мы последовательно:

**1. Загрузили и обработали все исходные таблицы:** Каждая таблица требовала индивидуального подхода к чтению и стандартизации. Все таблицы приведены к единому формату:

region | year | показатель

**2. Унифицировали наименования регионов:** Создан и применён расширенный словарь соответствий, устраняющий различия в написании регионов между источниками. Это позволило корректно объединить данные из всех таблиц.

**3. Привели данные к единому временному диапазону:** Для сопоставимости показателей были выбраны годы 2015-2020, как наиболее полно представленные во всех источниках.

**4. Выполнили объединение таблиц:** Все подготовленные таблицы объединены в единый датафрейм по ключам: region | year

**5. Добавили производные признаки** - важные социально-экономические индикаторы. Эти показатели позволяют перейти от «сырых» данных к содержательным характеристикам регионов.

**6. Сформировали итоговую исследовательскую таблицу:**

Полученный датафрейм содержит:
- демографические показатели,
- данные о доходах и бедности,
- показатели занятости,
- характеристики жилищных условий,
- промышленные, экономические и финансовые индикаторы,
- данные о преступности.

Он представляет собой целостный социально-экономический профиль каждого региона России в 2015-2020 гг.

**7. Экспортировали итоговый набор данных:** Финальный объединённый датасет сохранён в отдельный файл и готов к дальнейшему анализу.

### Итог:

Мы полностью завершили этап сбора, очистки и интеграции данных. Сформированный датасет является высококачественной основой для:
- разведывательного анализа данных (EDA),
- визуализации социальных и экономических различий регионов,
- построения моделей кластеризации,
- выявления территорий с характерными социально-экономическими профилями.

Следующая часть проекта будет посвящена исследовательскому анализу данных (EDA) и подготовке данных к кластеризации.